In [ ]:
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import copy
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score


lawinen_set = pd.read_csv('newData/gesamt_set.csv', sep=',')
lawinen_set['ID'] = lawinen_set['ID'].fillna(0)
avalanche_set = copy.copy(lawinen_set)

avalanche_set = avalanche_set.drop([ 'Lawinenabgänge', 'ID', 'Volumen', 'Lawinen_Art',], axis=1)
feature_names = avalanche_set.columns.drop('Avalanche')
X = avalanche_set[avalanche_set.columns.drop('Avalanche')]
y = avalanche_set['Avalanche']

X_train , X_test , y_train, y_test = train_test_split(X, y, random_state=0)

clf = make_pipeline(StandardScaler(), svm.SVC())
#clf = svm.SVC()
clf.fit(X_train,y_train)

#gram_test = np.dot(X_test, X_train.T)
pred = clf.predict(X_test)

print(accuracy_score(y_test, pred), cross_val_score(clf, X, y, cv=5))

In [ ]:
X = avalanche_set[['Lufttemperatur_7h', 'Schneehöhe']].to_numpy()
y = avalanche_set['Avalanche'].to_numpy()

h = .02  # step size in the mesh

# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
svc = svm.SVC(kernel='linear', C=C).fit(X, y)
preprocessed_svc = make_pipeline(StandardScaler(), svm.SVC(kernel='linear', C=C)).fit(X, y)
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(X, y)
poly_svc = svm.SVC(kernel='poly', degree=3, C=C).fit(X, y)
lin_svc = svm.LinearSVC(C=C).fit(X, y)

# create a mesh to plot in
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# title for the plots
titles = ['SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 3) kernel']


for i, clf in enumerate((svc, lin_svc, rbf_svc, poly_svc)):
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    plt.subplot(2, 2, i + 1)
    plt.subplots_adjust(wspace=0.4, hspace=0.4)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm)
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    plt.title(titles[i])

plt.show()